## This notebook is designed for the lecture on transport phenomena 

__As you will see, this notebook consists of four parts:__
- Real data for comparison
- A finite-differences implementation to solve the Partial Differential Equation (PDE)
- A cell to define the thermal diffusivity $\alpha$
- A comparison between real values and calculated ones

__First, choose a value for the thermal diffusivity, here called $\alpha$, for Section 2. Then, run all Sections 0 to 4 to get a result. If you want to change the parameter alpha afterwards, rerun Section 2-4 after changing alpha__

You can run the cells either by clicking on the cell and then on Run or press ctrl+enter


## 0. Install Packages

In this part, you will install the Packages numpy and matplotlib. Numpy is an essential package in python, used for large, multidimensional arrays. Matplotlib is a popular tool for visulazing data. Then, you will import these Packages. 

This might take a moment, but you have to execute this cell only once. 

In [1]:
# installation of Packages
!pip install numpy
!pip install matplotlib

# import of packages used here
import numpy as np # used for efficient vector computation
import matplotlib.pyplot as plt # used for plotting
import matplotlib.animation as animation # used for animation of plots
from IPython.display import HTML # used for showing animations 

Das System kann den angegebenen Pfad nicht finden.


Das System kann den angegebenen Pfad nicht finden.


## 1. Experimental Data

Below is the measured data during the experiment. Your goal will be to find the correct alpha value based on this data set

In [2]:
# Get Measured Data
# In this cell, the data points from the experiment are stored
# Please don't change anything and run this cell once


## 2. Thermal Diffusivity $\alpha$

### Set the thermal diffusivity, here given the variable name alpha, in the cell below. 

__Hint: Your value should be in the magnitude of $10^{-6}$__

In [3]:
# Parameter definition

# Define a value for alpha here :

###################################################################################################################

alpha = 10*10**-6  # thermal diffusivity [m²/s]

###################################################################################################################


## 3. Finite Differences Implementation 

### Differential Heat Equation 

$\Large \frac{\partial{T(x,y,t)}}{\partial{t}} = \alpha\ (\frac{\partial^2{T(x,y,t)}}{\partial^2{x}} + \frac{\partial^2{T(x,y,t)}}{\partial^2{y}} )$

To solve this partial differential equation (PDE), we need to define one initial condition (accounting for time) and four boundary conditions (accounting for space)

Initial condition:
- $ T(x\neq 0,y,t=0) = T_{initial} $ : At time $t=0$, the rest of the plat is cold

The plate is assumed to be adiabatic. Hence, the temperature function's change on the edges of the plate is zero. This can be formulated into the following boundary conditions:

Boundary conditions:
- $ \frac{\partial{T(x,y,t)}}{\partial{x}} = 0 \   $ for $ x=0 \ or \ x=w \ $ (w = plate's width) 

- $ \frac{\partial{T(x,y,t)}}{\partial{y}} = 0 \   $for  $ y=0 \ or \ x=h \ $ (h = plate's height) 

To account for the isothermal left side, we will furthermore define a constraint:

- $ T(x=0,y,t) = T_{hot} $ : We assume that at all time, the left edge of the plate is at temperature $T_{hot}$


### Finite Difference Discretization of Temperature equation
For a $x_n$, $y_n$ and time $t_n$, the temperature $T_{x_n,y_n}^{n+1}$ at the next time step $t_{n+1}$ can be approximated by:

$\Large T_{x_n,y_n}^{t_{n+1}} = T_{x_n,y_n}^{t_n} + \alpha \Delta t (\frac{T_{x_n,y_{n+1}}^{t_n} - 2 T_{x_n,y_{n}}^{t_n}+ T_{x_n,y_{n-1}}^{t_n}}{\Delta y^2} + \frac{T_{x_{n+1},y_{n}}^{t_n} - 2 T_{x_n,y_{n}}^{t_n} + T_{x_{n-1},y_{n}}^{t_n}}{\Delta x^2}) $

$x_{n+1},y_{n+1}$ and $x_{n-1},y_{n-1}$ are hereby the next and the former neighbour, respectively. 


In [4]:
# Physical parameter

#geometrical parameter
w = 20       # width of plate [mm]
h = 10       # height of plate [mm]

# temperatures
T_initial = 300   # initial temperature[K] (26.85°C)
T_hot = 700   # hot temperature [K] (426.85°C)

In [5]:
# simulation: in this cell the partial differential equation is solved through simulation

def do_timestep(T):
    # calculate each temperature(x,y,t) with central difference in space and Euler forward in time
    for i in range(dnframes):
        T[1:-1, 1:-1] = T[1:-1, 1:-1] + alpha * dt * (
              (T[2:, 1:-1] - 2*T[1:-1, 1:-1] + T[:-2, 1:-1])/dx2
              + (T[1:-1, 2:] - 2*T[1:-1, 1:-1] + T[1:-1, :-2])/dy2 )
        T[0,:] = T[1,:]    # set edge cases to same as neighbour
        T[-1,:] = T[-2,:]  # set edge cases to same as neighbour
        T[:,0] = T[:,1]    # set edge cases to same as neighbour
        T[:,-1] = T[:,-2]  # set edge cases to same as neighbour
        T = get_constraints(T.copy()) # implement constraints
    return T

def get_constraints(T):
    # introduce constrains on temperature function
    T[:,0] = T_hot # isothermal hot left side as constraint
    return T

    
### simulation parameter ###
dx = 0.1      # finite step in x direction [mm]
dy = 0.1      #finite step in y direction [mm]

#time
simulation_time = 50 #[min] 

#number of nodes
nx = int(w/dx) #number of nodes in x direction 
ny = int(h/dy) #number of nodes in y direction

dx2, dy2 = dx*dx, dy*dy  #[mm²]

# Number of timesteps
dt = dx2 * dy2 / (2 * alpha * (dx2 + dy2)) # discretized timestep [ms]
nsteps = int(simulation_time*60/dt*1000)

# Number of frames 
nframes = 40 #to keep computation feasable, only 40 frames are used, regardeless simulation time
dnframes = int(nsteps/nframes) #number of timesteps in a single frame


### Initialization ###

# initialization at time 0
T0 = T_initial * np.ones((nx, ny)) # initialize temperature function to all Cool
T0 = get_constraints(T0) #introduce constraints 

# initialization temperature function for all timesteps
T_tot = np.ones([nx,ny,nframes+1]) # initialize all to one (carried out to save time and allocate memory beforehand)
T_tot[:,:,0] = T0 # at timestep zero, equal to initialial temperature function 


### Simulation ###
#iterate through time
for i in range(nframes):
    global T
    T = T_tot[:,:,i] # current temperature function 
    T_tot[:,:,i+1]= do_timestep(T.copy()) 
    

In [6]:
# animate results

# helper functions for animation

def KtoC(T):
    # converts temperature funcion from Kelvin to Celsius
    conversionfactor = 273.15
    return (T-conversionfactor)

def init():
    # initializes figure and sets desgin of image
    global fig, ax, im
    fig = plt.figure(1)
    # image with cmap and color bar
    im = plt.imshow(KtoC(T_tot[:,:,0]),cmap=plt.get_cmap('hot'), vmin=KtoC(T_initial),vmax=KtoC(T_hot),extent=[0,w,h,0])
    plt.xlabel('[cm]')
    plt.ylabel('[cm]')
    cbar_ax = fig.add_axes([0.9, 0.15, 0.03, 0.7]) 
    cbar_ax.set_xlabel('$T$ / °C', labelpad=20)
    fig.colorbar(im, cax=cbar_ax)
    #initialize image with temperature at time 0
    im.set_data(T_tot[:,:,0])
    plt.close()
    return 

def animate(i):
    # changes image of temperature to time: i*dt 
    global fig, T_tot
    fig.suptitle('{:.1f} min'.format(simulation_time/nframes*i))  #set title above image to current time
    im.set_data(KtoC(T_tot[:,:,i])) #change temperature to i-th temperature
    return 

# execute animation
plt.ion()
init()
ani = animation.FuncAnimation(fig, animate, frames = nframes) 
HTML(ani.to_jshtml())

As you will see when you carried out the simulation correclty, with time the heat propagates from the left to the right. The propagation is at the beginning stronger because the temperature gradient is stronger. With time, it propagates more slowly

## 4. Results 

In [ ]:
# You will get your result when executing this cell. The result is the mean squared error, so the squarred error 
# between the model's predicted values and the measured values

#please fill out the google form with your alpha value and the obtained result 

# MSE:
#mse = (np.square(T - realT)).mean(axis=0)
#display('    ')
#display('The mean squarred error (MSE) for alpha = ' + str(alpha) + ' resulted in ' + str(np.round(mse,3)))

## Please submit your result to the google form for comparsion with the rest of the class

### Link: https://forms.gle/XjQnvJzamnYqvSBh6 